In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import trendy
import technical_indicators as ti
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline  
df_all = pd.read_csv("SPFB.RTS_150101_160331 (1).csv", usecols = ['<DATE>', '<CLOSE>', '<VOL>'], parse_dates = True, index_col = '<DATE>')

df = df_all['20150101':'20160801']
df['CHANGE'] = 1 - df['<CLOSE>'].shift(1)/ df['<CLOSE>']  #Set returns

#Set classes based on returns
df['CLASS'] = 0
df['CLASS'][df['CHANGE']< -0.01] = 1
df['CLASS'][df['CHANGE']> 0.01] = 2
print(df.head())
#Calculate technical indicators
ema = ti.ema(df['<CLOSE>'].values, 10)
bb = ti.bb(df['<CLOSE>'].values, 10)

df = df[9:]#Delete data without indicators
#Calculate indicators regardless price value
df['EMA_PRICE'] = (ema - df['<CLOSE>'])/df['<CLOSE>']
df['BB_UP_PRICE'] = (bb[:,0] - df['<CLOSE>'])/df['<CLOSE>']
df['BB_DOWN_PRICE'] = (bb[:,2] - df['<CLOSE>'])/df['<CLOSE>']

#Split test and train data
df_train = df['20150101':'20160101']
df_test = df['20160102':]

#Create regression classification model
knr = KNeighborsClassifier(n_neighbors=10)
knr.fit(df_train[['EMA_PRICE', 'BB_UP_PRICE', 'BB_DOWN_PRICE']], df_train['CLASS'])

#Predict classes
df_test['PREDICT'] = knr.predict(df_test[['EMA_PRICE', 'BB_UP_PRICE', 'BB_DOWN_PRICE']])
profit = df_test[(df_test['PREDICT'] == df_test['CLASS']) & (df_test['PREDICT'] != 0)]
print(df_test[(df_test['PREDICT'] == df_test['CLASS']) & (df_test['PREDICT'] != 0)])
print(df_test[(df_test['PREDICT'] != df_test['CLASS']) & (df_test['PREDICT'] != 0)])
print(len(df))
result = 1.0
for item in profit.values:
    if item[7] == 1:
        result = result * (1 - item[2])
    elif item[7] == 2:
          result = result * (1 + item[2])
print(result)

/Users/andreyponikar/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andreyponikar/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andreyponikar/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

            <CLOSE>  <VOL>    CHANGE  CLASS
<DATE>                                     
2015-01-05    72670  61859       NaN      0
2015-01-05    75280  42650  0.034671      2
2015-01-05    75850  30961  0.007515      0
2015-01-05    75030  20070 -0.010929      1
2015-01-05    75480  21941  0.005962      0
            <CLOSE>   <VOL>    CHANGE  CLASS  EMA_PRICE  BB_UP_PRICE  \
<DATE>                                                                 
2016-01-11    70620  107559 -0.043614      1   0.037733     0.067103   
2016-01-11    68990   94789 -0.011886      1   0.022106     0.033355   
2016-01-15    66210  159803 -0.025978      1   0.033789     0.078365   
2016-01-19    66740  135552  0.034013      2  -0.027238    -0.010366   
2016-01-20    63510  162293 -0.034325      1   0.028312     0.057101   
2016-01-21    61520  136811 -0.034460      1   0.021323     0.039300   
2016-01-21    64710  104188  0.022099      2  -0.026550    -0.004861   
2016-01-22    67900  138858  0.019293      2

/Users/andreyponikar/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
